In [81]:
import pandas as pd
import csv
import numpy
from glob import glob
label = glob("../dataset/label_r1/*csv")
cam = glob("../dataset/cam_r1/*csv")
print(len(label),len(cam))
del label[7]
del cam[7]

40 40


In [80]:
df_list = []
for a,b in zip(label,cam):
    try:
        df_a = pd.read_csv(a)
        df_b = pd.read_csv(b)
        df = pd.concat([df_a,df_b],axis=1)
        df_list.append(df)
    except:
        print(a)
        print(label.index(a))

../dataset/label_r1/20180614145705.label.csv
7


In [83]:
from collections import Counter
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
Acc= []
for train_index, test_index in kf.split(df_list):
    print(train_index,test_index)
    X,y=[],[]
    X_test,y_test=[],[]  
    for i in train_index:
        df_a = pd.read_csv(label[i])
        df_b = pd.read_csv(cam[i])
        df = pd.concat([df_a,df_b],axis=1)
        #df = df_list[i]
        df['U'] = df['U'].map(lambda x: 0 if x == "0" else 1)
        df['B'] = df['B'].map(lambda x: 0 if x == "0" else 1)
        df["A"] = df['A'].map(lambda x: 0 if x == "0" else 1)
        df['target'] = df['target'].map(lambda x: 0 if x == "A" else 1)
        df['action'] = df['action'].map({"None":0,"Continue":2,"Active":1,"Passive":3,"Nod":4})
        #dummy_df = pd.get_dummies(df)
        train_df = df.copy()
        from sklearn import preprocessing

        # 各カラムごとに正規化する
        """for column_name in train_df.columns:    
            # actionカラムは正規化しない
            if column_name in ['action',"target","U","A","B","utter_A","utter_B","utter_R"]:
                continue

            # 指定されたカラムを抽出
            train_column = train_df[column_name]
            #test_column = test_df[column_name]

            # trainを用いて平均と分散を計算する
            scaler = preprocessing.StandardScaler().fit(train_column.values.reshape(-1, 1))
            #  正規化
            train_df.loc[:, column_name] = scaler.transform(train_column.values.reshape(-1, 1))
            #test_df.loc[:, column_name] = scaler.transform(test_column.values.reshape(-1, 1))"""
        import numpy as np
        X_train = train_df.drop(['action', 'target',"U"], axis=1).values
        y_train = train_df['target'].values
        frame = 5
        for i in range(len(X_train)-frame):
            #if y_train[i+frame]==2:continue
            X.append(X_train[i:i+frame])
            y.append(y_train[i+frame])

    X = np.array(X)
    X=X.reshape(len(X),-1)
    print(X.shape)
    X = X.tolist()
    #for i in range(len(X)-1,-1,-1):
    #    if y[i] != 2: continue
    #    del X[i]
    #    del y[i]
    #print(y.count(2))

    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(class_weight="balanced",max_depth=2, n_estimators=100, n_jobs=-1, verbose=1)
    rf.fit(X, y)
    #class_weight="balanced",

    
    #for df in df_list[-5:]:
    for i in test_index:
        df_a = pd.read_csv(label[i])
        df_b = pd.read_csv(cam[i])
        df = pd.concat([df_a,df_b],axis=1)
        #df = df_list[i]
        df['U'] = df['U'].map(lambda x: 0 if x == "0" else 1)
        df['B'] = df['B'].map(lambda x: 0 if x == "0" else 1)
        df["A"] = df['A'].map(lambda x: 0 if x == "0" else 1)
        df['target'] = df['target'].map(lambda x: 0 if x == "A" else 1)
        df['action'] = df['action'].map({"None":0,"Continue":2,"Active":1,"Passive":3,"Nod":4})
        #dummy_df = pd.get_dummies(df)
        test_df = df.copy()
        from sklearn import preprocessing

        # 各カラムごとに正規化する
        """for column_name in train_df.columns:    
            # actionカラムは正規化しない
            if column_name in ['action',"target","U","A","B","utter_A","utter_B","utter_R"]:
                continue

            # 指定されたカラムを抽出
            train_column = train_df[column_name]
            #test_column = test_df[column_name]

            # trainを用いて平均と分散を計算する
            scaler = preprocessing.StandardScaler().fit(train_column.values.reshape(-1, 1))
            #  正規化
            train_df.loc[:, column_name] = scaler.transform(train_column.values.reshape(-1, 1))
            #test_df.loc[:, column_name] = scaler.transform(test_column.values.reshape(-1, 1))"""
        import numpy as np
        X_train = test_df.drop(['action', 'target',"U"], axis=1).values
        y_train = test_df['target'].values
        #print(len(X_train))

        frame = 5
        #print(X_train[0])
        for i in range(len(X_train)-frame):
            #if y_train[i+frame]==2:continue
            X_test.append(X_train[i:i+frame])
            y_test.append(y_train[i+frame])

    X_test = np.array(X_test)
    X_test=X_test.reshape(len(X_test),-1)
    print(X_test.shape)
    X_test = X_test.tolist()
    #for i in range(len(X_test)-1,-1,-1):
    #    if y_test[i] != 2: continue
    #    del X_test[i]
    #    del y_test[i]
    #print(y_test.count(2))

    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    #names = ["not Action","Active","Continue","Passive","Nod"]
    names = ["A","B"]
    print(classification_report(y_test, rf.predict(X_test),target_names=names))
    from sklearn.metrics import confusion_matrix
    #print(log_loss(y_test, rf.predict_proba(X_test)))
    Acc.append(accuracy_score(y_test,rf.predict(X_test)))
    print(confusion_matrix(y_test,rf.predict(X_test)))
print(np.mean(np.array(Acc)))

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38] [0 1 2 3]
(8662, 700)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.6s finished


(1059, 700)
             precision    recall  f1-score   support

          A       0.83      0.58      0.68       570
          B       0.64      0.86      0.73       489

avg / total       0.74      0.71      0.71      1059



[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


[[332 238]
 [ 70 419]]
[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32 33 34 35 36 37 38] [4 5 6 7]
(8031, 700)


...

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


[[292 115]
 [ 84 303]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35] [36 37 38]
(9359, 700)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


(362, 700)
             precision    recall  f1-score   support

          A       0.87      0.85      0.86       213
          B       0.80      0.82      0.81       149

avg / total       0.84      0.84      0.84       362

[[182  31]
 [ 27 122]]
0.7934617646019728


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [84]:
#df = df_list[0]
#df['U'] = df['U'].map(lambda x: 0 if x == "0" else 1)
#df['B'] = df['B'].map(lambda x: 0 if x == "0" else 1)
#df["A"] = df['A'].map(lambda x: 0 if x == "0" else 1)
#df['target'] = df['target'].map(lambda x: 0 if x == "A" else 1)
#df['action'] = df['action'].map({"None":0,"Continue":2,"Active":1,"Passive":3,"Nod":4})
#dummy_df = pd.get_dummies(df)
